In [1]:
# importing libraries
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
# get the website url
url = "https://turbo.az/autos"
marks, models, years, body_types, colors, capacities, hps, engine_types, transmissions, mileages, is_news, prices, price_currencies, links = [], [], [], [], [], [], [], [], [], [], [], [], [], []

# loop over pages
for i in range(1, 416):
    # get the page url
    link = url + f"?page={i}"
    response = requests.get(link)
    training_soup = BeautifulSoup(response.text, "html.parser")
    # loop over car ads
    for car in training_soup.find_all('div', attrs={"products-i"}):
        # get the car's link
        car_link = "https://turbo.az/" + car.find('a').get('href')
        car_response = requests.get(car_link)

        content = car_response.text
        car_soup = BeautifulSoup(content, "html.parser")

        # parsing the mark
        mark = car_soup.find_all('div', attrs={"product-properties__i"})[1].text
        mark_pattern = r'\bMarka'
        mark = re.split(mark_pattern, mark)[1]

        # parsing the model
        model = car_soup.find_all('div', attrs={"product-properties__i"})[2].text
        model_pattern = r'\bModel'
        model = re.split(model_pattern, model)[1]

        # parsing the year of production
        year = car_soup.find_all('div', attrs={"product-properties__i"})[3].text
        year_pattern = r'\bBuraxılış ili'
        year = re.split(year_pattern, year)[1]

        # parsing the body type
        body_type = car_soup.find_all('div', attrs={"product-properties__i"})[4].text
        body_type_pattern = r'\bBan növü'
        body_type = re.split(body_type_pattern, body_type)[1]

        # parsing the color
        color = car_soup.find_all('div', attrs={"product-properties__i"})[5].text
        color_pattern = r'\bRəng'
        color = re.split(color_pattern, color)[1]

        # parsing the attributes
        attrs = car_soup.find_all('div', attrs={"product-properties__i"})[6].text
        attrs_pattern = r'\bMühərrik'
        attrs = re.split(attrs_pattern, attrs)[1]
        attrs = re.split('/', attrs)

        engine_capacity = re.findall(r"[0-9]+.[0-9]*", attrs[0])[0]
        hp = re.findall(r"[0-9]+", attrs[1])[0]
        engine_type = attrs[2]

        # parsing the transmission type
        transmission = car_soup.find_all('div', attrs={"product-properties__i"})[8].text
        transmission_pattern = r"\bSürətlər qutusu"
        transmission = re.split(transmission_pattern, transmission)[1]

        # parsing the mileage of the car
        mileage = car_soup.find_all('div', attrs={"product-properties__i"})[7].text
        mileage = mileage.replace(" ", "")
        mileage_pattern = r'[0-9]+'
        mileage = int(re.findall(mileage_pattern, mileage)[0])

        # parsing the status of the car (is it new or used)
        is_new = car_soup.find_all('div', attrs={"product-properties__i"})[10].text
        is_new_pattern = r'\bYeni'
        is_new = re.split(is_new_pattern, is_new)[1]

        # parsing the price and price currency
        price_full = car_soup.find('div', attrs={'product-price__i product-price__i--bold'}).text
        price_full = price_full.replace(" ", "")
        price_pattern = r'[0-9]+'
        price = int(re.findall(price_pattern, price_full)[0])
        price_currency = re.split(price_pattern, price_full)[1]

        marks.append(mark)
        models.append(model)
        years.append(year)
        body_types.append(body_type)
        colors.append(color)
        capacities.append(engine_capacity)
        hps.append(hp)
        engine_types.append(engine_type)
        transmissions.append(transmission)
        mileages.append(mileage)
        is_news.append(is_new)
        prices.append(price)
        price_currencies.append(price_currency)
        links.append(car_link)

        # making the dataframe
        data_list = list(zip(marks, models, years, body_types, colors, capacities, hps, engine_types, transmissions, mileages, is_news, prices, price_currencies, links))
        column_names = ['Mark', 'Model', 'Year', 'Body_type', 'Color', 'Capacity', 'HorsePowers', 'EngineType', 'Transmission', 'Mileage', 'Is_new', 'Price', 'Price_currency', 'Link']
        data = pd.DataFrame(data_list, columns=column_names)

# write dataframe to csv file
data.to_csv("cars.csv")